In [0]:
#@title Params
 

document_parser_lib_version='1.1.14'#@param {type:"string"}
charter_filename='/content/\u0413\u041F\u041D \u0423\u0441\u0442\u0430\u0432.docx'#@param {type:"string"}
# _git_branch = 'protocols-4'
_git_branch = 'charters-charity'#@param {type:"string"}

# INIT

## pull code

In [0]:

import json
import subprocess
import sys
import time

import numpy as np
from IPython.core.display import display, HTML
from google.colab import files

!pip install overrides

Ничто = None



def exec(x):
  r = subprocess.check_output(x, shell=True)
  r = r.decode('unicode-escape').encode('latin1').decode('utf8')
  print(r)


print(f"fetching code from GitHub.....{_git_branch}")
try:
  exec('rm -r nlp_tools')
except:
  pass
exec(f'git clone --single-branch --branch {_git_branch} https://github.com/nemoware/analyser.git nlp_tools')

print('🦊 GIT revision:')
exec('cd nlp_tools\ngit rev-list --reverse HEAD | awk "{ print NR }" | tail -n 1\ngit branch\ngit log -3 --pretty=%B')

sys.path.insert(0, 'nlp_tools')

print('❤️importing Code from GitHub ... DONE')



 
  

In [0]:
!pip install pyjarowinkler

### Init document-parser lib

In [0]:
%tensorflow_version 1.13.1

lib_version = document_parser_lib_version
import os
if not os.path.isfile(f'document-parser-{lib_version}-distribution.zip'):
  !wget https://github.com/nemoware/document-parser/releases/download/$lib_version/document-parser-$lib_version-distribution.zip
if not os.path.isdir(f'document-parser-{lib_version}'):
  !unzip document-parser-$lib_version-distribution.zip

 
os.environ ['documentparser']=f'/content/document-parser-{lib_version}'
from integration.word_document_parser import WordDocParser, join_paragraphs
wp = WordDocParser()

### imports

In [0]:
import os
import pickle
import unittest

import numpy as np

from analyser.contract_parser import ContractAnlysingContext, ContractDocument
from analyser.contract_patterns import ContractPatternFactory
from analyser.legal_docs import LegalDocument
 
from analyser.ml_tools import *

# from headers_detector import doc_features, load_model, make_headline_attention_vector
from analyser.hyperparams import HyperParameters
from analyser.protocol_parser import protocol_votes_re
 

## render tools

In [0]:
#----
import matplotlib as mpl
from analyser.documents import TextMap
from colab_support.renderer import HtmlRenderer
from analyser.legal_docs import DocumentJson
 

class DemoRenderer(HtmlRenderer):
  def render_color_text(self, tokens, weights, colormap='coolwarm', print_debug=False, _range=None, separator=' '):
    html = self.to_color_text(tokens, weights, colormap, print_debug, _range, separator=separator)
    display(HTML(html))

  def to_color_text(self, tokens, weights, colormap='coolwarm', print_debug=False, _range=None, separator=' '):
    return super()._to_color_text(tokens, weights, mpl, colormap=colormap, _range=_range, separator=separator)

   
renderer_ = DemoRenderer()

def print_json_summary(cd:DocumentJson):
  wordsmap = TextMap(cd.normal_text, cd.tokenization_maps['$words'])
  print(f'read file {cd.filename}')

  for tag in cd.tags:
    span = tag.span
    _map = cd.tokenization_maps[tag.span_map]
    print(tag)
    
def json2html(cd:DocumentJson):
  wordsmap = TextMap(cd.normal_text, cd.tokenization_maps['words'])
  markup_vector = np.zeros(len(wordsmap))
 
  for hd in cd.headers:    
    span = hd['span']      
    markup_vector[span[0]:span[1]] += 2

  for tag in cd.attributes:
    if 'span' in  cd.attributes[tag]:
      span = cd.attributes[tag]['span']
      confidence = cd.attributes[tag]['confidence']
      markup_vector[span[0]:span[1]] += confidence
    else:
      warnings.warn(f'{tag} has no span')

  return renderer_.to_color_text(wordsmap.tokens, markup_vector)

def print_headers(d: LegalDocument):
  for p in d.paragraphs:
    print('\t --> 📂', d.substr(p.header))

## 💅 Init Embedder(s)

In [0]:
# from protocol_parser import  ProtocolPatternFactory
from tf_support.embedder_elmo import ElmoEmbedder
# from contract_patterns import ContractPatternFactory
elmo_embedder = ElmoEmbedder()
elmo_embedder_default = ElmoEmbedder(layer_name="default")

# protocols_factory = ProtocolPatternFactory(elmo_embedder)
# contracts_factory = ContractPatternFactory(elmo_embedder)

In [0]:
 
from analyser.legal_docs import tokenize_doc_into_sentences_map, ContractValue
from analyser.ml_tools import *
from analyser.parsing import ParsingContext
from analyser.patterns import *
 
from analyser.text_tools import *

# legal_docs.py
from tf_support.embedder_elmo import ElmoEmbedder
 



# Charter parsing

## classes

In [0]:
from enum import Enum, unique, EnumMeta
from typing import List
from analyser.structures import DisplayStringEnumMeta, CharterSubject
from analyser.legal_docs import embedd_sentences, LegalDocumentExt
from analyser.charter_parser import competence_headline_pattern_prefix


In [0]:

from analyser.embedding_tools import AbstractEmbedder
from analyser.legal_docs import map_headlines_to_patterns
from analyser.charter_parser import embedd_charter_subject_patterns, get_charter_subj_attentions, CharterParser, CharterDocument
from analyser.contract_parser import find_value_sign_currency_attention
 

def collect_sentences_having_constraint_values(subdoc: LegalDocumentExt, contract_values: [ContractValue]):
  # collect sentences having constraint values
  unique_sentence_spans = []
  for contract_value in contract_values:
    contract_value_sentence_span = subdoc.tokens_map.sentence_at_index(contract_value.parent.span[0],
                                                                        return_delimiters=False)
    if contract_value_sentence_span not in unique_sentence_spans:
      unique_sentence_spans.append(contract_value_sentence_span)
  # --
  unique_sentence_spans = merge_colliding_spans(unique_sentence_spans, eps=1)
  return unique_sentence_spans

class CharterParser2(CharterParser):
  def __init__(self, embedder: AbstractEmbedder = None, elmo_embedder_default: AbstractEmbedder = None):
    super().__init__(embedder, elmo_embedder_default)
  
  

  def find_attributes(self, charter: CharterDocument) -> CharterDocument:

    if charter.sentences_embeddings is None:
      # lazy embedding
      self._ebmedd(charter)

    # reset for preventing doubling tags
    charter.margin_values = []
    charter.constraint_tags = []
    charter.charity_tags = []
    charter.org_levels = []
    charter.org_level_tags = []
    # --------------
    # (('Pattern name', 16), 0.8978644013404846),
    patterns_by_headers = map_headlines_to_patterns(charter,
                                                    self.patterns_named_embeddings,
                                                    self.elmo_embedder_default)

    _parent_org_level_tag_keys = []
    for p_mapping in patterns_by_headers:
      # kkk += 1

      _paragraph_id = p_mapping[0][1]
      _pattern_name = p_mapping[0][0]

      paragraph_body = charter.paragraphs[_paragraph_id].body
      confidence = p_mapping[1]
      _org_level_name = _pattern_name.split('/')[-1]
      org_level: OrgStructuralLevel = OrgStructuralLevel[_org_level_name]
      subdoc = charter.subdoc_slice(paragraph_body.as_slice())

      parent_org_level_tag = SemanticTag(f"{org_level.name}", org_level.name, paragraph_body.span)
      parent_org_level_tag.confidence = confidence

      constraint_tags, values, subject_attentions_map = self.attribute_charter_subjects(subdoc, self.subj_patterns_embeddings,
                                                                parent_org_level_tag)
      for value in values:
        value += subdoc.start  # TODO: move into attribute_charter_subjects

      for constraint_tag in constraint_tags:
        constraint_tag.offset(subdoc.start)  # TODO: move into attribute_charter_subjects

      charter.margin_values += values  # TODO: collect all, then assign to charter
      charter.constraint_tags += constraint_tags

      if values:
        _key = parent_org_level_tag.get_key()
        if _key in _parent_org_level_tag_keys:  # number keys to avoid duplicates
          parent_org_level_tag.kind = _key + f"-{len(_parent_org_level_tag_keys)}"
        charter.org_levels.append(parent_org_level_tag)  # TODO: collect all, then assign to charter
        _parent_org_level_tag_keys.append(_key)

      charity_subj_av_words = subject_attentions_map[CharterSubject.Charity]['words']
      charity_tag = find_charity_paragraphs(parent_org_level_tag, subdoc, charity_subj_av_words )
      print('-----charity_tag', charity_tag)
      if charity_tag is not None:
        charter.charity_tags.append(charity_tag)

    return charter

 


  def attribute_charter_subjects(self, subdoc: LegalDocumentExt, emb_subj_patterns, parent_org_level_tag: SemanticTag):
    """
    :param subdoc:
    :param emb_subj_patterns:

          emb_subj_patterns[subj] = {
            'patterns':patterns,
            'embedding':patterns_emb
          }

    :return:
    """

    # ---------------
    subject_attentions_map = get_charter_subj_attentions(subdoc, emb_subj_patterns)
    contract_values: [ContractValue] = find_value_sign_currency_attention(subdoc, None)
    # -------------------

    # collect sentences having constraint values
    unique_sentence_spans = collect_sentences_having_constraint_values(subdoc, contract_values)

    # attribute sentences to subject
    constraint_tags = []

    for sentence_number, contract_value_sentence_span in enumerate(unique_sentence_spans, start=1):

      max_confidence = 0
      best_subject: CharterSubject = CharterSubject.Other

      for subj in subject_attentions_map.keys():
        av = subject_attentions_map[subj]['words']

        confidence_region = av[contract_value_sentence_span[0]:contract_value_sentence_span[1]]
        confidence = estimate_confidence_by_mean_top_non_zeros(confidence_region)

        if confidence > max_confidence:
          max_confidence = confidence
          best_subject = subj

      #
      constraint_tag = SemanticTag(SemanticTag.number_key(best_subject.name, sentence_number),
                                   best_subject.name, contract_value_sentence_span,
                                   parent=parent_org_level_tag)
      constraint_tag.confidence = max_confidence
      constraint_tags.append(constraint_tag)

      # nest values
      for contract_value in contract_values:
        if constraint_tag.is_nested(contract_value.parent.span):
          contract_value.parent.set_parent_tag(constraint_tag)

      self._rename_margin_values_tags(contract_values)

    return constraint_tags, contract_values, subject_attentions_map


 

###-------------------
charter_parser = CharterParser2(elmo_embedder, elmo_embedder_default)
results = wp.read_doc(charter_filename)
# results = wp.read_doc('/content/ЕЮ Устав.docx')

for d in results['documents'][:1]:  # XXX
  if 'CHARTER' == d['documentType']:    
    charter = join_paragraphs(d, 'no_id')    

print_headers(charter)

In [0]:
from analyser.parsing import AuditContext
## PHASE 1
charter_parser.find_org_date_number(charter, AuditContext())

In [0]:
## PHASE 2
from analyser.contract_parser import _find _most_relevant_paragraph, find_value_sign_currency_attention
def find_charity_paragraphs(parent_org_level_tag: SemanticTag, subdoc: LegalDocument,
                            charity_subject_attention: FixedVector) -> SemanticTag:
  paragraph_span, confidence, paragraph_attention_vector = _find_most_relevant_paragraph(subdoc,
                                                                                         charity_subject_attention,
                                                                                         min_len=20,
                                                                                         return_delimiters=False)

  if confidence > HyperParameters.charter_charity_attention_confidence:
    subject_tag = SemanticTag(CharterSubject.Charity.name, CharterSubject.Charity.name, paragraph_span,
                              parent=parent_org_level_tag)
    subject_tag.offset(subdoc.start)
    subject_tag.confidence = confidence
    return subject_tag


charter_parser.find_attributes(charter)

In [0]:

if False:
  import matplotlib.pyplot as plt
  import seaborn as sns
  plt.figure(num=None, figsize=(12, 6), dpi=80, facecolor='w', edgecolor='k')
  sns.set(font_scale=1)
  ax = sns.heatmap(globals()['header_to_pattern_distances_'],  fmt="", annot_kws={"size": 8}, vmin=0, vmax=1, yticklabels=charter.headers_as_sentences())
  plt.show()

In [0]:

renderer_.render_color_text(charter.tokens_cc, charter.get_tags_attention() , _range=(0,5) )

# Tests

### constraint_tags

In [0]:
for tag in charter.org_levels:
  print (tag.get_parent())
  print (tag)

  sub = charter [tag.as_slice()]
  renderer_.render_color_text(sub.tokens_cc, sub.get_tags_attention() , _range=(-5,5) )

### Tags

In [0]:
for tag in charter.get_tags():
  print (tag)

## Find org level sections
 - find all headers spans
 - compare headers with patterns 
 - find interresting spans

In [0]:
header_slices = [p.header.as_slice() for p in charter.paragraphs]
header_slices

remap to sentences

In [0]:
header_slices_sent = charter.tokens_map.remap_slices(header_slices, charter.sentence_map)
header_slices_sent


In [0]:
section_ends = [ x[0] for x in header_slices_sent ] + [ len(charter.sentence_map) ] #+end of doc
section_ends

Available sentence pattern (names)

In [0]:
charter.distances_per_sentence_pattern_dict.keys()

In [0]:
# fi 
# av_competence_headline_pattern = max_exclusive_pattern_by_prefix(charter.distances_per_sentence_pattern_dict, competence_headline_pattern_prefix)
# av_competence_headline_pattern = relu(av_competence_headline_pattern, 0.66)

# av_interresting_headers = headers_att * av_competence_headline_pattern
# #words map
# # sections_spans_words = charter.sentence_map.remap_slices(v, charter.tokens_map)
 
# # v = charter.distances_per_sentence_pattern_dict['headline.head.directors.n']# - 0.1*charter.distances_per_sentence_pattern_dict['headline.head.directors.not'] 

# renderer_.render_color_text(charter.sentence_map.tokens, av_interresting_headers, _range=(0,1), separator='¶<br>')
# # renderer_.render_color_text(charter.tokens_cc, sections_spans_words, _range=(0,1) )

In [0]:
charter_parser.patterns_dict

## charter_subjects_attribution

In [0]:
# del get_charter_subj_attentions

from analyser.legal_docs import remap_attention_vector
from analyser.contract_parser import _find_most_relevant_paragraph, find_value_sign_currency_attention
from analyser.legal_docs import remap_attention_vector, ContractValue, LegalDocumentExt
from analyser.ml_tools import *

 
from analyser.charter_parser import find_charity_paragraphs, get_charter_subj_attentions



 

###DEBUG attribute_charter_subjects -------------------


charter.margin_values = []
charter.constraint_tags=[]
# --------------
filtered = [p_mapping for p_mapping in patterns_by_headers if p_mapping]
for p_mapping in filtered:
  print('-'*40)
  
  paragraph = p_mapping[4]
  org_level_name = p_mapping[1].split('/')[-1]
  org_level = OrgStructuralLevel[org_level_name]
  print(org_level_name)
  subdoc = charter.subdoc_slice(paragraph.body.as_slice())
  parent_org_level_tag = SemanticTag(org_level.name, org_level, paragraph.body.span)

  constraint_tags, values = charter_parser.attribute_charter_subjects(subdoc, charter_parser.subj_patterns_embeddings,
                                                            parent_org_level_tag)
  
  charter.margin_values += values
  charter.constraint_tags += constraint_tags
  for c in constraint_tags:
    print(c)

# for p_mapping in patterns_by_headers:
#   print()
#   print()
#   if p_mapping:
#     print('p_mapping', p_mapping)
#     paragraph =  p_mapping[4]
#     org_level_name =  p_mapping[1].split('/')[-1]
#     org_level = OrgStructuralLevel[org_level_name]

#     subdoc = charter.subdoc_slice( paragraph.body.as_slice())
#     parent_org_level_tag = SemanticTag(org_level.name, org_level, paragraph.body.span)

#     constraint_tags, values = charter_parser.attribute_charter_subjects(subdoc, self.subj_patterns_embeddings, parent_org_level_tag)

#     print(org_level)
#     print('constraint_tags')
#     for c in constraint_tags:
#       print(c)
#     print('values')

for c in charter.margin_values:
  print(c.value)
  print ('quote:', charter.substr (c.parent))

In [0]:
charter.margin_values=[]
for p_mapping in patterns_by_headers:
  if p_mapping:
    # paragraph =  p_mapping[4]
    # org_level =  p_mapping[1]

    paragraph =  p_mapping[4]
    org_level_name =  p_mapping[1].split('/')[-1] # TODO FIX it, looks weird
    org_level = OrgStructuralLevel[org_level_name]

    
    subdoc = charter.subdoc_slice( paragraph.body.as_slice())
    parent_org_level_tag = SemanticTag(org_level.name, org_level, paragraph.body.span)

    consent_patterns_distances = calc_distances_per_pattern_dict(subdoc.sentences_embeddings,
                                                                consent_patterns,
                                                                consent_patterns_emb)
    
    # charity_patterns_distances = calc_distances_per_pattern_dict(subdoc.sentences_embeddings,
    #                                                             consulting_patterns,
    #                                                             consulting_patterns_emb)
    
    # print(order_patterns_distances.keys())
    values:List[ContractValue] = find_value_sign_currency_attention(subdoc, None)
    numbers_attention = get_values_attention(subdoc, values)

    consent_v = relu(max_exclusive_pattern_by_prefix(consent_patterns_distances, 'consent/'),0.5)
    consent_words = remap_attention_vector(consent_v, subdoc.sentence_map, subdoc.tokens_map)

    # charity_subj_av = relu(max_exclusive_pattern_by_prefix(charity_patterns_distances, 'consulting/'),0.6)
    # charity_subj_av_words = remap_attention_vector(charity_subj_av, subdoc.sentence_map, subdoc.tokens_map)
    # order_patterns_distances['order/consent']
    # renderer_.render_color_text(subdoc.sentence_map.tokens, v, _range=(0,1), separator='😱 <br>')
    
    # charity_patterns_distances_words = remap_attention_vector(v, subdoc.sentence_map, subdoc.tokens_map)

    
    
    # dd = get_charter_subj_attentions(subdoc, emb_subj_patterns)
    # charity_subj_av_words = dd[CharterSubject.Charity]['words']
    subject_attentions_map = get_charter_subj_attentions(subdoc, emb_subj_patterns)

    charity_subj_av_words = subject_attentions_map[CharterSubject.Charity]['words']
    charity_tag = find_charity_paragraphs(parent_org_level_tag, subdoc, (charity_subj_av_words+consent_words)/2)
    print(charity_tag)
    if charity_tag is not None:
      charter.charity_tags.append(charity_tag)

 

    constraint_tags, values = attribute_charter_subjects(subdoc, emb_subj_patterns, parent_org_level_tag)
    charter.margin_values += values

    # renderer_.render_color_text(subdoc.tokens_cc, charity_subj_av_words + numbers_attention , _range=(0,2) )
    

In [0]:
renderer_.render_color_text(charter.tokens_cc, charter.get_tags_attention() , _range=(0,2) )

### Test attribution

In [0]:


for p_mapping in patterns_by_headers:
  if p_mapping:
    # paragraph =  p_mapping[4]
    # org_level =  p_mapping[1]

    paragraph =  p_mapping[4]
    org_level_name =  p_mapping[1].split('/')[-1]
    org_level=OrgStructuralLevel[org_level_name]

    
    subdoc = charter.subdoc_slice( paragraph.body.as_slice())
    
    consent_patterns_distances = calc_distances_per_pattern_dict(subdoc.sentences_embeddings,
                                                                consent_patterns,
                                                                consent_patterns_emb)
    
     
    

    consent_v = relu(max_exclusive_pattern_by_prefix(consent_patterns_distances, 'consent/'),0.5)
    consent_words = remap_attention_vector(consent_v, subdoc.sentence_map, subdoc.tokens_map)
     
     
    values:List[ContractValue] = find_value_sign_currency_attention(subdoc, None)

    #collect sentences with constraint values
    sentence_spans=[]
    for value in values:
      sentence_span = subdoc.tokens_map.sentence_at_index(value.parent.span[0], return_delimiters=True)      
      if sentence_span not in sentence_spans:
        sentence_spans.append(sentence_span) 
    # sentence_spans = merge_colliding_spans(sentence_spans, eps=1)
     


    subject_attentions_map = get_charter_subj_attentions(subdoc, emb_subj_patterns)

    # attribute sentences to subject
    
    for span in sentence_spans:
      print ( )
      print ('-'*30)
      print ('VALUE', sentence_span, subdoc.tokens_map.text_range(span))
      for subj in subject_attentions_map.keys():
        av = subject_attentions_map[subj]['words']
        
        confidence_region = av[span[0]:span[1]]
        confidence = estimate_confidence_by_mean_top_non_zeros(confidence_region)

        renderer_.render_color_text(subdoc.tokens_map[span[0]:span[1]], confidence_region , _range=(0,1) )
        print(subj,  confidence )
      print ('~'*30)
    numbers_attention = get_values_attention(subdoc, values)


    

    charity_subj_av_words = subject_attentions_map[CharterSubject.Consulting]['words']
    charity_tag = find_charity_paragraphs(org_level, subdoc,(charity_subj_av_words+consent_words)/2)
    print(charity_tag)
    
    



# Save json

In [0]:
json_str=charter.to_json()

json_obj = DocumentJson.from_json(json_str)
with open('charter.json', 'w') as file: 
  file.write(json_str)